In [ ]:
import sys
sys.path.append('Y:\\GoogleDrive\\Coding\\best_fit_modules')
import monitor_tools_wb

import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import pickle
import re
import numpy as np
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, Text, DateTime, Float, Date

chrome_driver_path = 'C:\chromedriver_win32\chromedriver.exe'
postgre_user_engine = 'postgresql+psycopg2://user:password@ip:port/db_name'

pd.set_option('display.max_columns', None)

## Получение информации по нашим товарам в наличии (airflow)

In [ ]:
# получение нашего стока из базы
sql_query = '''
                SELECT
                    "nmId" AS nm_id,
                    article AS article
                FROM wb_products_info
                WHERE stock > 0
                UNION
                SELECT
                    "nmId" AS nm_id,
                    "supplierArticle" AS article
                FROM wb_stat_stocks
                WHERE quantity > 0
            '''

engine = create_engine(postgre_user_engine, echo=True)
try:
    our_stock = pd.read_sql(sql_query, con=engine)
finally:
    engine.dispose()

our_stock['url'] = 'https://www.wildberries.ru/catalog/' + our_stock.nm_id + '/detail.aspx'

our_stock_with_price = monitor_tools_wb.parse_product_cards_data(our_stock, chrome_driver_path, param_json_src=True)

our_stock_with_price_and_json_data = monitor_tools_wb.parse_product_cards_data_json(our_stock_with_price)

with open('wb_files/pkl/wb_our_stock_data.pkl', 'wb') as file:
    pickle.dump(our_stock_with_price_and_json_data, file)

## Получение списка товаров по списку урлов категорий (airflow)

In [ ]:
wb_category_url_list = [
            'https://www.wildberries.ru/catalog/sport/sportivnoe-pitanie?sort=popular&page=1',
            'https://www.wildberries.ru/catalog/pitanie/napitki?sort=popular&page=1&xsubject=3382%3B3441',
            'https://www.wildberries.ru/catalog/pitanie/sneki?sort=popular&page=1&xsubject=3014',
            'https://www.wildberries.ru/catalog/pitanie/sladosti/shokolad-i-shokoladnye-batonchiki?sort=popular&page=1&xsubject=2878',
            'https://www.wildberries.ru/catalog/pitanie/sladosti/shokolad-i-shokoladnye-batonchiki?sort=popular&page=1&xsubject=3568',
            'https://www.wildberries.ru/catalog/produkty/dobavki-pishchevye?sort=popular&page=1',
            'https://www.wildberries.ru/catalog/produkty/sladosti/orehi?sort=popular&page=1&xsubject=3407',
            'https://www.wildberries.ru/catalog/produkty/sladosti/pasty?sort=popular&page=1',
            'https://www.wildberries.ru/catalog/dom-i-dacha/zdorove/vitaminy-i-bady?sort=popular&page=1',
            'https://www.wildberries.ru/catalog/zdorove/lechebnoe-pitanie-?sort=popular&page=1'
]

prod_list = monitor_tools_wb.get_product_list_from_cat_pages(wb_category_url_list, chrome_driver_path, 14000)

with open('wb_files/pkl/wb_prod_list.pkl', 'wb') as file:
    pickle.dump(prod_list, file)

## Получение информации из джейсонов карточек товара (airflow)

In [ ]:
with open('wb_files/pkl/wb_prod_list.pkl', 'rb') as file: 
    prod_list = pickle.load(file)
    
prod_list_with_json_data = monitor_tools_wb.parse_product_cards_data_json(prod_list)

with open('wb_files/pkl/wb_prod_list_data.pkl', 'wb') as file:
    pickle.dump(prod_list_with_json_data, file)

# Сопоставление (пока manual)

#### Функции и словари для задачи

In [ ]:
# функция для перевода строковых значений разного формати в int
def to_int(x):
    formatted_value = re.sub("\..*", "", x)
    formatted_value = re.sub("\/.*", "", formatted_value)
    formatted_value = re.sub("[^0-9]", "", formatted_value)
    
    if formatted_value == '':
        return 0

    return int(formatted_value)

In [ ]:
# словарь вариантов написания бренда
brand_variations_dict = {
                  #'ActivLab': [],
                  #'APPLIED NUTRITION': []
                  #'BCN': [],
                  'be first': ['be first1', 'befirst', 'Be first'],
                  'Beauty Fit': ['BeautyFit'],
                  'Bio Schwartz': ['bioschwartz'],
                  'BombBar': ['Bombbar Chikalab', 'Bombbar (Бомббар)', 'Бомббар "BOMBBAR"', 'Bombbar.', 'BomBar', 'Бомббар'],
                  'Bounty': ['Баунти'],
                  'BSN': ['BSN Amino X'],
                  'Bulk Supplements': ['Bulk'],
                  'California Gold Nutrition': ['California Gold Nutrition iHerb', 'California Gold Nutrition USA', 
                                                'California Gold жидкий витамин D3 для детей', 'California Gold Омега-3 с витамином D3 для детей', 'Iherb California Gold Collagen',
                                                'Калифорния Голд Нутришн', 'California Gold', 'California', 'Калифорния'],
                  'Carlson Labs': ['Carlson Labs D3 детский', 'Carlson'],
                  'FIT KIT': ['FITKIT', 'fitkit', 'fit kit'],
                  'CHIKALAB': ['CHIKALAB.', 'CHIKA LAB', 'chikalab', 'chika lab'],
                  #'GEL4U': [],
                  'IronMan': ['Iron Man-QB', 'Iron Man.', 'Iron Man'],
                  #'Isopure': [],
                  #'Lactoflorene': [],
                  'Lake Avenue Nutrition': ['ASL/ Lake Avenue Nutrition', 'Lake Avenue'],
                  "M&M's": ["M&M's United Kingdom", "M&Ms Europe", "M&Ms", "M&M,s"],
                  'Mars': ['Mars United Kingdom', 'Mars protein', 'Mars Hi Protein bars', 'Mars Incorporated', 'Mars Chocolate Drinks & Treats', 'Mars Inc.'],
                  'MAXLER': ['Maxler EU', 'Maxler USA', 'MAXLER Аминокислоты 100% Pure Glutamine, Maxler', 'MaxlerGamine', 'Maxler Acetyl l-Carnitine', 'Maxler Zinc Picolinate',
                             'Maxler Omega-3 Gold', 'Протеин Maxler 100% Golden Whey Шоколад', 'Maxler Creatine Nari', 'HOT/ Сывороточный протеин Maxler 100%'],
                  'MD': ['MD.', 'MD-QB'],
                  'Milky Way': ['MilkyWay', 'MilkyWay United Kingdom', 'Milky Way USA', 'Milky way chocopasta'],
                  #'Multipower': [],
                  #'MUSCLETECH': [],
                  'MyProtein': ['My Protein'],
                  #'Natrol': [],
                  #'Novo': [],
                  'NOW': ['NOW FOODS', 'SuperNOW', 'NOW / NOW FOODS', 'NOW FOODS / NOW', 'NOWFOODS', 'NOW Foods, NOW', 'NOW FOOD', 'NOW FOODS/НАУ ФУДС', 'NOW/NOW FOODS',
                          'NOW Omega 3', 'NOW Foods, жирные кислоты омега-3, 200 капсул', 'NOW Foods GABA 750 мг 100 капсул', 'NOW FOODS iHerb', 'NOW FOODS ДУБЛЬ', 'now sports',
                          'НАУ', 'Now Витамин Д3 5000 120 капсул', 'Рыбий жир и омега Now Foods 500ЭПК/250ГК'],
                  'NUTREX': ['Nutrex research', 'Nutrex BCAA DRIVE Аминокислоты 200 таблеток', 'Nutrex (USA)'],
                  'Olimp Sport Nutrition': ['Olimp'],
                  #'OstroVit': [],
                  #'PhD': [],
                  #'Power System': [],
                  #'Protego': [],
                  #'Real Pharm': [],
                  'Science in sports (SIS)': ['Science in Sport (SIS)', 'SIS (Science in Sport)', 'Sis', 'Science in Sports', 'Science in sport'],
                  'SFD': ['SFD Nutrition'],
                  #'SmartLife': [],
                  #'Snaq Fabriq': [],
                  'Snickers': ['Snickers шоколадный батончик молочный, нуга', 'Snickers СНИКЕРС', 'Сникерс,Snickers', 'Snickers/ Шоколадный батончик/ молочный шоколад',
                               'Шоколад Snickers', 'Snickers шоколадный батончик молочный', 'Сникерс'],
                  'SOLGAR': ['SOLGAR СОЛГАР', 'SOLGAR 1947', 'SOLGAR витамины', 'SOLGAR/СОЛГАР бета-каротин', 'SOLGAR. Омега -3. 50 капсул', 'SOLGAR/Солгар',
                             'SOLGAR/Солгар для женщин', 'Солгар Solgar', 'Solgar Inc', 'СОЛГАР ВИТАМИНЫ', 'Solgar', 'Солгар', 'СОЛГАР'],
                  #'Sporty': [],
                  'Swanson': ['Swanson (USA)'],
                  #'The Gummies Co': [],
                  #'UNS': [],
                  'USPLabs': ['USP Labs'],
                  #'V-Shape Supps': [],
                  #'VITUAL LABORATORIES': [],
                  'Warrior Crunch': ['Warrior'],
                  #'WOW ENERGY': [],
                  'XXI Power': ['XXIPower'],
                  'YAMAMOTO': ['Yamamoto Kanpo']
                }

In [ ]:
#data_wb.loc[data_wb['selling.brand_name'].str.contains('California')]['selling.brand_name'].value_counts()

In [ ]:
# список списков для фильтрации по правилу: 
# если в названии встечается одно из подсписка и сравниваемом названии также встречается что-то из этого подсписка, они считаются одинаковыми товарами
#['', ''],
name_similarity_both_true = [
    ['arginin', 'аргинин'], ['alcar', 'алкар'], ['astaxanthin', 'астаксантин'], 
    ['bacop', 'бакоп'], ['bcaa', 'бцаа'], ['biotin', 'биотин'], ['bromelain', 'бромелаин'], 
    ['cardio', 'кардио'], ['carnitin', 'карнитин'], ['carnosin', 'карнозин'], ['casein', 'казеин'], ['cholin', 'холин'], ['chelat', 'хелат'], 
    ['cla ', 'cla,', 'linoleic acid', 'линолевая кислота'], ['collagen', 'коллаген'], ['curcumin', 'куркумин', 'turmeric'], ['coq10', 'coenzyme q10', 'коэнзим q10'],
    ['d3', 'd 3', 'д3', 'д 3'], ['dmae', 'дмае'], 
    ['echinacea', 'эхинацея'], ['electrolyte drinc', 'напиток с электролитами', 'напиток электролитный'],
    ['gaba', 'гамк'], ['glucosamin', 'глюкозамин'], ['glutathion', 'глутатион'], ['grape seed', 'виноградных косточек'], ['guarana', 'гуарана'], 
    ['htp', 'хтп', 'гидрокситриптофан', 'гидрокси триптофан'], ['huperzine', 'гуперзин'], ['immune 4', 'иммуне 4'], ['iron', 'железо'],
    ["johns wort", "john's wort", "зверобо"],
    ['folic', 'фолиев'], ['fiber', 'клетчатка'], ['k2', 'k 2', 'к2', 'к 2'], 
    ['lactobif', 'лактобиф'], ['lysin', 'лизин'], ['life boost', 'лайф буст'], ['lipoic', 'липоев'], ['liposomal', 'липосомал'], ['lutein', 'лютеин'],
    ['maca', 'мака'], ['мелатонин', 'melatonin'], ['menopaus', 'менопауз'], ['niacin', 'ниацин'], 
    ['olive leaf', 'оливковых листьев', 'листьев оливы'], ['omega', 'омега'], 
    ['picolinat', 'пиколинат'], ['prebiotic', 'пребиотик'], ['pqq', 'pyrroloquinoline', 'пирролохинолинхинон'],
    ['sambucus', 'бузина', 'бузины'], ['selenium', 'селен'], ['spirulin', 'спирулин'], ['superfood', 'суперфуд', 'супер фуд'], 
    ['theanin', 'теанин'], ['thistle', 'чертополох', 'расторопша'], ['tyrosin', 'тирозин'], ['quercetin', 'кверцетин'], 
    ['vitamin c', 'витамин ц', 'витамин с', 'витамин c'],
    ['vitamin b', 'витамин b', 'витамин б', 'витамин в', 'витамины группы b', 'витамины группы б', 'витамины группы в'],
    ['vitamin d3', 'витамин d3', 'витамин д3', 'vitamin d-3', 'витамин d-3', 'витамин д-3', 'vitamin d 3', 'витамин d 3', 'витамин д 3'],
    ['vitamin e', 'витамин е', 'витамин e'],
    ['zinc orotat', 'оротат цинка'], ['zma']
]
#['whey', 'вей', 'протеин'],

#### Загрузка данных

In [ ]:
with open('wb_files/pkl/wb_prod_list_data.pkl', 'rb') as file: 
    data_wb = pickle.load(file)
    
with open('wb_files/pkl/wb_our_stock_data.pkl', 'rb') as file: 
    data_our = pickle.load(file)
    
print(data_wb.shape, data_our.shape)

#### Предобработка

In [ ]:
%%time

engine = create_engine(postgre_user_engine, echo=True)

# удаление из data_wb товаров, которые уже на мониторинге
query = '''
            SELECT id
            FROM monitor_mp_prices
        '''
ids_in_monitoring = pd.read_sql(query, con=engine)
data_wb.drop(data_wb[data_wb.nm_id.isin(ids_in_monitoring.id)].index, inplace=True)

# удаление наших товаров из списка товаров конкурентов
data_wb.drop(data_wb[data_wb.nm_id.isin(data_our.nm_id)].index, inplace=True)

# замена nan пустую строку
columns_with_str_nan = ['selling.brand_name', 'Страна производства', 'Вкус']
for col_name in columns_with_str_nan:
    data_our[col_name].fillna('', inplace=True)
    data_wb[col_name].fillna('', inplace=True)

# дроп строк, где не хватает важных данных
data_our.dropna(subset=['imt_name'], inplace=True)
data_wb.dropna(subset=['imt_name'], inplace=True)

# замена ; на , (это для нормального отображения данных в csv)
for column in ['Вкус', 'imt_name']:
    data_our[column] = data_our[column].apply(lambda x: x.replace(';', ','))
    data_wb[column] = data_wb[column].apply(lambda x: x.replace(';', ','))

# приведение названий брендов к единому формату (используется словарь определенный ранее)
for key in brand_variations_dict:
    for name in brand_variations_dict[key]:
        data_our['selling.brand_name'] = data_our['selling.brand_name'].apply(lambda x: x.replace(name, key) if x == name else x)
        data_wb['selling.brand_name'] = data_wb['selling.brand_name'].apply(lambda x: x.replace(name, key) if x == name else x)

data_wb['selling.brand_name'] = data_wb['selling.brand_name'].apply(lambda x: 'California Gold Nutrition' if 'California Gold' in x else x)
        
# приведение данных в колонках важных для сопоставления к единому числовому формату с помощью ранее написанной функции to_int (оставляет в колонках только цифры, если цифр нет, то 0)
search_columns = ['Количество предметов в упаковке', 
                  'Вес товара с упаковкой (г)',
                  'Вес товара без упаковки (г)',
                  'Ширина упаковки',
                  'Высота упаковки',
                  'Глубина упаковки',
                  'Пищевая ценность белки',
                  'Пищевая ценность жиры',
                  'Пищевая ценность углеводы',
                  'Энергетическая ценность калории (на 100 гр.)']

for col_name in search_columns:
    if col_name in data_our.columns:
        data_our[col_name].fillna('0', inplace=True)
        data_our[col_name] = data_our[col_name].apply(to_int)
        
    if col_name in data_wb.columns:
        data_wb[col_name].fillna('0', inplace=True)
        data_wb[col_name] = data_wb[col_name].apply(to_int)
        
engine.dispose()

#### Сопоставление

In [ ]:
%%time

# удаление из wb_data колонок, которые не будут участвовать в сопоставлении
search_lake = data_wb[['nm_id', 
                       'imt_name',
                       'selling.brand_name',
                       'price_with_disc',
                       'Количество предметов в упаковке',
                       'Вкус',
                       'url']]

# первый этап сопоставление товаров по параметрам
results_1 = pd.DataFrame()

for i in range(data_our.shape[0]):
    
    # фильтр по бренду
    search_pool = search_lake[search_lake['selling.brand_name'] == data_our.iloc[i]['selling.brand_name']]
    
    # фильтр по количеству в упаковке
    if data_our.iloc[i]['Количество предметов в упаковке'] != 0:
        search_pool = search_pool[(search_pool['Количество предметов в упаковке'] == data_our.iloc[i]['Количество предметов в упаковке']) 
                                  | (search_pool['Количество предметов в упаковке'] == 0)]
    
    # фильтр по цене (+-30% и еще сверху +-50 рублей, чтобы нормально учитывать разницу у товаров с низкими ценами)
    if data_our.iloc[i]['price_with_disc'] != 0:
        search_pool = search_pool[(search_pool['price_with_disc'] <= (int(data_our.iloc[i]['price_with_disc'] * 1.3) + 50)) & 
                                  (search_pool['price_with_disc'] >= (int(data_our.iloc[i]['price_with_disc'] * 0.7) - 50))]

    # добавление к каждой строке нашего соответствующего товара и присоединение к результирующему датасету
    search_pool['our_article'] = data_our.iloc[i]['article']
    search_pool['our_nm_id'] = data_our.iloc[i]['nm_id']
    search_pool['our_url'] = data_our.iloc[i]['url']
    search_pool['our_name'] = data_our.iloc[i]['imt_name']
    search_pool['our_price'] = data_our.iloc[i]['price_with_disc']

    results_1 = results_1.append(search_pool)
    results_1.reset_index(inplace=True)
    results_1.drop(columns='index', inplace=True)
    
print(results_1.shape)

In [ ]:
%%time

# удаление тех пар, которые уже сопоставлялись до этого
engine = create_engine(postgre_user_engine)
query = '''
            SELECT *
            FROM monitor_wb_already_compared_pairs
        '''
already_compared = pd.read_sql(query, con=engine)

for i in range(already_compared.shape[0]):
    print(i)
    row_to_delete = results_1[(results_1['our_nm_id'] == already_compared['our_nm_id'][i]) & (results_1['nm_id'] == already_compared['nm_id'][i])]
    if not row_to_delete.empty:
        results_1.drop(index=row_to_delete.index[0], inplace=True)
        print(row_to_delete.index[0])

results_1.reset_index(inplace=True)
results_1.drop(columns='index', inplace=True)

engine.dispose()

In [ ]:
%%time

# второй этап фильтрации (по вхождениям подстрок в оба названия)
results_1['name_similarity_index'] = 0
results_1['searchable_name'] = False

for i in range(results_1.shape[0]):
    searchable_name = False
    similarity_index = 0
    
    for sublist in name_similarity_both_true:
        our_name_contains = any((word in results_1['our_name'][i].lower() for word in sublist))
        comp_name_contains = any((word in results_1['imt_name'][i].lower() for word in sublist))
        
        if our_name_contains:
            searchable_name = True
        if our_name_contains and comp_name_contains:
            similarity_index += 1
            break
            
    results_1['name_similarity_index'][i] = similarity_index
    results_1['searchable_name'][i] = searchable_name
        
results_2 = results_1[(results_1['searchable_name'] == False) | 
                      ((results_1['searchable_name'] == True) & (results_1['name_similarity_index'] > 0))]

In [ ]:
# сохранение в csv (для финальной ручной сверки)
results_2[['our_article',
            'our_nm_id',
            'our_name',
            'our_price',
            'our_url',
            'nm_id',
            'imt_name',
            'Количество предметов в упаковке',
            'Вкус',
            'price_with_disc',
            'url',
            'selling.brand_name',
            'name_similarity_index',
            'searchable_name']].to_csv('wb_files/csv/results_2.csv', encoding='utf-8-sig', sep='|', index=False)

In [ ]:
# датафрейм с товарами, которые отсеял фильтр (для проверки корректности его работы)
results_2_false = results_1[((results_1['searchable_name'] == True) & (results_1['name_similarity_index'] == 0))]

## Загрузка результатов ручной сверки в бд с предварительным обновлением цены и продавца (manual)

In [ ]:
# загрузка эксель файла с результатами сверки
search_results_final = pd.read_excel('wb_files/wb_search_results_final.xlsx', dtype={'our_article': 'str',
                                                                            'our_nm_id': 'str',
                                                                            'nm_id': 'str'})

search_results_final = search_results_final[['our_article', 'our_nm_id', 'our_name', 'our_price', 'our_url', 'nm_id', 'imt_name', 'price_with_disc', 'url']]

search_results_final['mp'] = 'WB'

search_results_final.rename(columns={'nm_id': 'id', 
                                     'our_nm_id': 'our_id',
                                     'imt_name': 'name'}, inplace=True)

# актуализация цены товаров
search_results_final_with_seller = monitor_tools_wb.parse_product_cards_data(search_results_final, 
                                                                             chrome_driver_path, 
                                                                             param_seller_name=True)

# загрузка в таблицу в бд, которая используется для bi
engine = create_engine(postgre_user_engine)

search_results_final_with_seller.to_sql(
         'monitor_mp_prices',
         engine,
         if_exists='append',
         index=False)

engine.dispose()

## Загрузка в бд пар id товаров, которые уже были сопоставлены (manual)

In [ ]:
# в отдельную эксельку из файла, в котором проводилась сверка, выносятся два столбца: наш id товара и id товара конкурента, который скрипт сопоставления ему сопоставил
# эти пары загружаются в базу данных, чтобы при следующей сверке их не учитывать

already_compared_pairs = pd.read_excel('wb_files/wb_already_compared_pairs.xlsx', dtype={'our_nm_id': 'str', 'nm_id': 'str'})

engine = create_engine(postgre_user_engine)

already_compared_pairs.to_sql(
         'monitor_wb_already_compared_pairs',
         engine,
         if_exists='append',
         index=False)

engine.dispose()